In [1]:
import pandas as pd
import numpy as np

## Preliminary Back-of-the-Envelope Calculation

1. Calculate Cost of 1 ad (per person) in 2012
2. Calculate cost of 22 ads for each voting-age person in 2012
3. Calculate how much 100k will get you.

In [2]:
#
# Preliminary back of the envelope calculation:
# 

cost_of_all_ads_in_2012 = 366 * 10**6 #
voters_in_2012 = 206 * 10**6 # we assume that we can successfully target voting-age people
total_ads = (30 + 45) * voters_in_2012
average_cost_of_ad = cost_of_all_ads_in_2012 / total_ads
print(f"average cost to show 1 voting-age-person an ad : {round(average_cost_of_ad,5)}$")
cost_to_show_an_ad_to_everyone = (voters_in_2012 * average_cost_of_ad)


print(f"cost to show 1 ad to everyone : {cost_to_show_an_ad_to_everyone} $") # ~107 Million $

potential_conversion_rate = (1-.5) # assume that 1/2 of pop are dems, and you want to convert the rest
conversions = voters_in_2012 * potential_conversion_rate* (0.01/100) # ~100k potential people converted
print(f"potential conversions after everyone sees 1 ad: {int(conversions)}")

# if ~100M$ converts 227k voters, then 100k$ (1000 times smaller), should convert ~210 people
conversions_for_100k = (100000/cost_to_show_an_ad_to_everyone) *conversions
print(f"conversions after 100k of ads : {int(conversions_for_100k) }")


average cost to show 1 voting-age-person an ad : 0.02369$
cost to show 1 ad to everyone : 4880000.0 $
potential conversions after everyone sees 1 ad: 10300
conversions after 100k of ads : 211


## Data-Driven Back-of-the-Envelope Calculation

In [3]:
# helper functions to parse strings into ints
def dollars_to_int(df, name):
    if (df[name].dtype != np.int64):
        df[name]=  df[name].apply(lambda x: "".join(x.lstrip()[1:].split(','))).astype(np.int64)

def number_to_int(df, name):
    if (df[name].dtype != np.int64):
        df[name]=  df[name].apply(lambda x: "".join(x.lstrip().split(','))).astype(np.int64)
    

In [4]:
# clean up dataframe

path = "Release20_mapdata.csv" # data collected from Wesleyan Media Project at https://www.dropbox.com/s/ap3msdzenjkizuq/Release20_mapdata.xlsx?dl=0
# referenced here : https://mediaproject.wesleyan.edu/releases-102920/#table3 (in the paper https://www.kellogg.northwestern.edu/faculty/spenkuch/research/advertising.pdf)

wesleyan_dma_df = pd.read_csv(path)
wesleyan_dma_df["dma_name"]=wesleyan_dma_df['Media Market'].str.lower()
dollars_to_int(wesleyan_dma_df, 'Est. Cost 9Apr20 to 25Oct29') 
number_to_int(wesleyan_dma_df, "Total Airings 9Apr20 to 25Oct20")
dollars_to_int(wesleyan_dma_df, 'Est. Cost 12Oct20 to 25Oct20')
number_to_int(wesleyan_dma_df, "Total Airings 12Oct20 to 25Oct20")
wesleyan_dma_df["dma_name"]= wesleyan_dma_df["dma_name"].astype(str)

# compute average cost of airing a single ad
wesleyan_dma_df['average_cost_per_airing 9Apr20 to 25Oct20'] =  wesleyan_dma_df['Est. Cost 9Apr20 to 25Oct29']/ wesleyan_dma_df['Total Airings 9Apr20 to 25Oct20']
wesleyan_dma_df['average_cost_per_airing 12Oct20 to 25Oct20'] = wesleyan_dma_df['Est. Cost 12Oct20 to 25Oct20']/ wesleyan_dma_df['Total Airings 12Oct20 to 25Oct20']

In [5]:
wesleyan_dma_df

,Media Market,Market State,dma,Total Airings 9Apr20 to 25Oct20,Est. Cost 9Apr20 to 25Oct29,Pro-Biden Airings 9Apr20 to 25Oct20,Pro-Trump Airings 9Apr20 to 25Oct20,Biden Ad Adv 9Apr20 to 25Oct20,Total Airings 12Oct20 to 25Oct20,Est. Cost 12Oct20 to 25Oct20,Pro-Biden Airings 12Oct20 to 25Oct20,Pro-Trump Airings 12Oct20 to 25Oct20,Biden Ad Adv 12Oct20 to 25Oct20,dma_name,average_cost_per_airing 9Apr20 to 25Oct20,average_cost_per_airing 12Oct20 to 25Oct20
0,ABILENE-SWEETWATER,TX,662,38,5020,13,25,-12,15,2890,13,2,11,abilene-sweetwater,132.105263,192.666667
1,"ALBANY, GA",GA,525,6325,803140,719,"5,606","-4,887",1691,269520,620,"1,071",-451,"albany, ga",126.978656,159.384979
2,"ALBANY, NY",NY,532,94,4000,85,9,76,94,4000,85,9,76,"albany, ny",42.553191,42.553191
3,ALBUQUERQUE,NM,790,1109,205980,74,"1,035",-961,92,12150,72,20,52,albuquerque,185.734896,132.065217
4,ALEXANDRIA,LA,644,12,380,10,2,8,12,380,10,2,8,alexandria,31.666667,31.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,WILMINGTON,NC,550,12295,2570680,"6,048","6,247",-199,1471,428250,788,683,105,wilmington,209.083367,291.128484
209,YAKIMA,WA,810,130,5000,27,103,-76,33,870,27,6,21,yakima,38.461538,26.363636
210,YOUNGSTOWN,OH,536,3124,680170,"3,067",57,"3,010",608,149840,606,2,604,youngstown,217.724072,246.447368
211,YUMA-EL CENTRO,AZ,771,7047,1053230,"6,541",506,"6,035",1774,184540,"1,766",8,"1,758",yuma-el centro,149.457925,104.024803


In [6]:
#
# get population size information, to join the other DMA information
#
pop_size_path = "mediatracks_dma_pop_size.txt" # data pulled from https://mediatracks.com/resources/nielsen-dma-rankings-2021/
pop_size_df = pd.read_csv(pop_size_path, delimiter="\t",header = None)

# reorganize the Dataframe to have column names and to be easy for us to join on
pop_size_df= pop_size_df.drop(columns=[3])
pop_size_df = pop_size_df.rename(columns = {0:"dma", 1:"dma_name",2:"tv homes"})
number_to_int(pop_size_df, "tv homes") # cast population to int
pop_size_df= pop_size_df.set_index("dma") # set index 
pop_size_df["dma_name"]=pop_size_df['dma_name'].str.lower() # change the city names to lower-case



In [7]:
#
# join the 2 dataframes on city-name
#

ave_people_per_home = 2.52 # https://www.statista.com/statistics/183648/average-size-of-households-in-the-us/
dma_df = wesleyan_dma_df.join(pop_size_df.set_index('dma_name'), on = "dma_name")
dma_df['cost_per_airing_per_person'] = dma_df['average_cost_per_airing 12Oct20 to 25Oct20']/(dma_df['tv homes'] * ave_people_per_home) # there are on average 2.52 
dma_df

,Media Market,Market State,dma,Total Airings 9Apr20 to 25Oct20,Est. Cost 9Apr20 to 25Oct29,Pro-Biden Airings 9Apr20 to 25Oct20,Pro-Trump Airings 9Apr20 to 25Oct20,Biden Ad Adv 9Apr20 to 25Oct20,Total Airings 12Oct20 to 25Oct20,Est. Cost 12Oct20 to 25Oct20,Pro-Biden Airings 12Oct20 to 25Oct20,Pro-Trump Airings 12Oct20 to 25Oct20,Biden Ad Adv 12Oct20 to 25Oct20,dma_name,average_cost_per_airing 9Apr20 to 25Oct20,average_cost_per_airing 12Oct20 to 25Oct20,tv homes,cost_per_airing_per_person
0,ABILENE-SWEETWATER,TX,662,38,5020,13,25,-12,15,2890,13,2,11,abilene-sweetwater,132.105263,192.666667,104440.0,0.000732
1,"ALBANY, GA",GA,525,6325,803140,719,"5,606","-4,887",1691,269520,620,"1,071",-451,"albany, ga",126.978656,159.384979,130950.0,0.000483
2,"ALBANY, NY",NY,532,94,4000,85,9,76,94,4000,85,9,76,"albany, ny",42.553191,42.553191,NaN,NaN
3,ALBUQUERQUE,NM,790,1109,205980,74,"1,035",-961,92,12150,72,20,52,albuquerque,185.734896,132.065217,NaN,NaN
4,ALEXANDRIA,LA,644,12,380,10,2,8,12,380,10,2,8,alexandria,31.666667,31.666667,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,WILMINGTON,NC,550,12295,2570680,"6,048","6,247",-199,1471,428250,788,683,105,wilmington,209.083367,291.128484,190390.0,0.000607
209,YAKIMA,WA,810,130,5000,27,103,-76,33,870,27,6,21,yakima,38.461538,26.363636,NaN,NaN
210,YOUNGSTOWN,OH,536,3124,680170,"3,067",57,"3,010",608,149840,606,2,604,youngstown,217.724072,246.447368,213380.0,0.000458
211,YUMA-EL CENTRO,AZ,771,7047,1053230,"6,541",506,"6,035",1774,184540,"1,766",8,"1,758",yuma-el centro,149.457925,104.024803,101040.0,0.000409


In [8]:
dma_df.describe()

,dma,Total Airings 9Apr20 to 25Oct20,Est. Cost 9Apr20 to 25Oct29,Total Airings 12Oct20 to 25Oct20,Est. Cost 12Oct20 to 25Oct20,average_cost_per_airing 9Apr20 to 25Oct20,average_cost_per_airing 12Oct20 to 25Oct20,tv homes,cost_per_airing_per_person
count,214.000000,214.000000,2.140000e+02,214.000000,2.140000e+02,214.000000,214.000000,1.160000e+02,116.000000
mean,641.140187,4977.443925,3.767904e+06,1002.275701,9.949012e+05,425.929278,457.150981,5.165839e+05,0.000688
std,114.196680,10359.299118,1.108276e+07,1969.443384,3.042489e+06,1318.030845,1293.148927,9.479913e+05,0.001732
min,-1.000000,3.000000,1.200000e+02,2.000000,1.200000e+02,18.846154,10.714286,3.590000e+03,0.000029
25%,552.250000,37.250000,2.517500e+03,20.000000,1.142500e+03,60.572917,53.353270,9.084750e+04,0.000102
50%,627.500000,132.500000,1.737000e+04,52.000000,6.015000e+03,142.120867,110.224561,2.142250e+05,0.000215
75%,724.750000,3531.250000,1.110228e+06,812.000000,3.387825e+05,414.811100,470.772321,5.308525e+05,0.000520
max,999.000000,53911.000000,9.767437e+07,9009.000000,2.574914e+07,17798.589700,17048.198758,7.100300e+06,0.013266


### Let's compute a ballpark for how far 100k$ will get us.

1. sort the DMAs by population
2. While you have money : pick the next DMA, and show N (let's pick 21) ads to everyone in the DMA
3. For each DMA, calculate the persuadable people and multiply that by the persuasion rate.


In [9]:
# helper variables + function
def is_nan(n): # return true if is Nan
    return n != n

# 
# Let's compute how far 100k$ will get us, if we are trying to show each DMA 21 more ad spots (and shift the distribution of votes by 0.5%). 
# This is only to get a ball-park estimate.
#

# voter turnout ~60%  https://en.wikipedia.org/wiki/Voter_turnout_in_United_States_presidential_elections
# percent of population over the age of 18 is ~74% https://www.infoplease.com/us/census/demographic-statistics
voting_population_percentage = 0.74 * 0.6
dem_population = 0.5 # population you are interested in converting # crudely say that 1/2 of pop is dem and half is rep.
persuasion_rate_per_ad = 0.01 / 100 # let's lower the persuasion rate of dems and reps to the same, at 0.01% per ad
hours_of_tv_watched = 5 # https://www.nielsen.com/wp-content/uploads/sites/3/2019/04/Global-Video-Report-How-People-Watch-1.pdf


ads_to_show = 21 # an arbitrary number

In [10]:
def persuade_as_many_as_can(df, cash):
    """ given a dataframe of all the DMAs, try to persuade as many as you can, 
    starting with the first one.
        1. pop off a DMA, 
        2. Pay for N (21) ads and show the population of that DMA those ads
        3. keep a running total of the expected_persuaded_people, computed by voting-age-population-in-DMA *persuasion-rate * N
    Arguments:
        df [pd.Dataframe] : dataframe of DMAs, in the row order that you wish to show Ads to. 
        cash [int] : available cash
    returns : 
        potential-persuaded-population [int]
    """
    pop_shift = 0
    for i, row in df.iterrows(): # for each DMA
        if is_nan(row['tv homes']): # skip DMA if you are missing population data
            continue
        # cost to show N ads to the entire DMA
        cost_of_ads = ads_to_show * row['average_cost_per_airing 12Oct20 to 25Oct20']
        cash -= cost_of_ads

        if cash<0:
            break
        # number of people persuadable = (ppl in DMA) * (voting age pop) * (people not already voting for you) 
        persuadable_people = row['tv homes'] *  ave_people_per_home *(voting_population_percentage * (1-dem_population)) 
        # persuaded people = persuadable people * persusaion rate * ads shown * probability of seeing the ad.
        expected_persuaded_people = persuadable_people * persuasion_rate_per_ad *ads_to_show * (hours_of_tv_watched/24)
        pop_shift += expected_persuaded_people
    
    print(f"Potential persuaded voters with 100,000$ is: {int(pop_shift)}")
    return pop_shift
    

### Let's experiment a little bit with different kinds of ways to allocate the money
1. The most populous DMAs first 
2. The least populous DMAs first
3. The most cost-efficient ads first (Where cost-efficiency = ave-cost-of-airing / homes)
4. The most cost-efficient ads first (Where cost-efficiency = ave-cost-of-airing / homes)

In [11]:

cash = 100000 # original amount of cash.

dma_df = dma_df.sort_values(by='tv homes', ascending = False)
print("1. Distribute money to TV-Ad spending by DMA based on the *most populous* first:")
persuade_as_many_as_can(dma_df, cash)
print("-----")

print("2. Distribute money to TV-Ad spending by DMA based on the *least populous* first:")
dma_df = dma_df.sort_values(by='tv homes', ascending = True)
persuade_as_many_as_can(dma_df, cash) ;

dma_df['cost efficiency'] = dma_df['average_cost_per_airing 12Oct20 to 25Oct20']/ dma_df['tv homes'] # dollars per airing per home
print("-----")

print("3. Distribute money to TV-Ad spending by DMA based on the *most cost-efficient* first:")

dma_df = dma_df.sort_values(by='cost efficiency', ascending = True)
persuade_as_many_as_can(dma_df, cash) ;
print("-----")

print("4. Distribute money to TV-Ad spending by DMA based on the *least cost-efficient* first:")
dma_df = dma_df.sort_values(by='cost efficiency', ascending = False)
persuade_as_many_as_can(dma_df, cash) ;


1. Distribute money to TV-Ad spending by DMA based on the *most populous* first:
Potential persuaded voters with 100,000$ is: 4514
-----
2. Distribute money to TV-Ad spending by DMA based on the *least populous* first:
Potential persuaded voters with 100,000$ is: 311
-----
3. Distribute money to TV-Ad spending by DMA based on the *most cost-efficient* first:
Potential persuaded voters with 100,000$ is: 7101
-----
4. Distribute money to TV-Ad spending by DMA based on the *least cost-efficient* first:
Potential persuaded voters with 100,000$ is: 44
